In [71]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [72]:
import re
import html
from html.parser import unescape

In [73]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/radik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/radik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/radik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [74]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

In [75]:
import pickle

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [127]:
filename = 'output_tweets'
infile = open(filename,'rb')
combine_df = pickle.load(infile)
infile.close()

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.

Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.

Исключим стоп-слова с помощью stop_words='english'.

Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().


In [131]:
labels, texts_tweet_token = [], []
for i, line in enumerate(combine_df['tweet_stemmed']):
    texts_tweet_token.append(" ".join(line))

count_vectorizer = CountVectorizer(stop_words='english', max_features=1000, max_df=0.9)
bag_of_words = count_vectorizer.fit_transform(texts_tweet_token)
feature_names = count_vectorizer.get_feature_names()
df_cv_tweet_token = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

filename = 'df_cv_tweet_token'
outfile = open(filename,'wb')
pickle.dump(df_cv_tweet_token,outfile)
outfile.close()

In [132]:
labels, texts_tweet_lemmatized = [], []
for i, line in enumerate(combine_df['tweet_lemmatized']):
    texts_tweet_lemmatized.append(" ".join(line))

count_vectorizer = CountVectorizer(stop_words='english', max_features=1000, max_df=0.9)
bag_of_words = count_vectorizer.fit_transform(texts_tweet_lemmatized)
feature_names = count_vectorizer.get_feature_names()
df_cv_tweet_lemmatized = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

filename = 'df_cv_tweet_lemmatized'
outfile = open(filename,'wb')
pickle.dump(df_cv_tweet_lemmatized,outfile)
outfile.close()

2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). 
Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.

Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.

Исключим стоп-слова с помощью stop_words='english'.

Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().


In [133]:
tfidfVectorizer = TfidfVectorizer(stop_words='english', max_features=1000, max_df=0.9)
bag_of_words = tfidfVectorizer.fit_transform(texts_tweet_token)
feature_names = tfidfVectorizer.get_feature_names()
df_tfv_tweet_token = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

filename = 'df_tfv_tweet_token'
outfile = open(filename,'wb')
pickle.dump(df_tfv_tweet_token,outfile)
outfile.close()

In [134]:
tfidfVectorizer = TfidfVectorizer(stop_words='english', max_features=1000, max_df=0.9)
bag_of_words = tfidfVectorizer.fit_transform(texts_tweet_lemmatized)
feature_names = tfidfVectorizer.get_feature_names()
df_tfv_tweet_lemmatized = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

filename = 'df_tfv_tweet_lemmatized'
outfile = open(filename,'wb')
pickle.dump(df_tfv_tweet_lemmatized,outfile)
outfile.close()

3. Натренируем gensim.models.Word2Vec модель на наших данных.

Тренировать будем на токенизированных твитах combine_df['tweet_token']

Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.

Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

In [78]:
# !pip install gensim
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count

In [84]:
# from gensim.models import Word2Vec
dataset = combine_df['tweet_token']
data = [d for d in dataset]
model = Word2Vec(data, size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34)

In [85]:
model.train(data, total_examples=model.corpus_count, epochs=20)

(9142951, 11726520)

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. 

Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. 

То же самое попробуем со словом "trump".

In [89]:
result = model.similar_by_word("dinner", topn=3)
print(result)

[('bihdaydinner', 0.563190221786499), ('cookout', 0.5501049757003784), ('shawarma', 0.5325543880462646)]


In [90]:
result = model.similar_by_word("trump", topn=3)
print(result)

[('donald', 0.5565332174301147), ('dumptrump', 0.5349440574645996), ('crony', 0.5135934948921204)]


In [105]:
result = model.most_similar(positive=['trump'])
print(result)

[('donald', 0.5565332174301147), ('dumptrump', 0.5349440574645996), ('crony', 0.5135934948921204), ('conman', 0.5072427988052368), ('donaldtrump', 0.5065755844116211), ('trumptrain', 0.5065163969993591), ('commie', 0.5063521862030029), ('impeachment', 0.5044173002243042), ('fuhered', 0.5018693208694458), ('delegaterevolt', 0.5010116100311279)]


5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).

Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [116]:
# определим схожесть между словами
similarity = model.similarity('food', 'burger')
print(similarity)

0.37801754


6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.

Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. 

В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.

На выходе должен получиться wordvec_df.shape = (49159, 200).


In [161]:
def get_vec(words):
    size = 200
    vec = np.zeros(size) 
    vec = vec.reshape(1, size)
    norm = 0
    for word in list(words):
        try:
            vec += model[word].reshape(1, size)
            norm += 1
        except KeyError:
            norm += 0
    if norm > 0:
        vec = vec / norm
    return vec

In [162]:
wordvec_df = np.array(list(map(lambda s: get_vec(s), combine_df['tweet_token'])))

In [166]:
wordvec_df = wordvec_df.reshape(49159, 200)
wordvec_df.shape

(49159, 200)

In [168]:
wordvec_df[0]

array([ 0.07023581,  0.20633666,  0.21011429, -0.12263244,  0.13387704,
        0.11778077, -0.2788592 ,  0.17378253, -0.3868844 ,  0.30214327,
       -0.04302077,  0.09093152,  0.09837542, -0.24681592, -0.17835215,
       -0.06520925, -0.00440741,  0.14072396, -0.25762915, -0.09013469,
        0.07741308,  0.01606197, -0.15052575,  0.29240547, -0.04468606,
       -0.11161922, -0.22644533,  0.10272843,  0.14724459, -0.1788101 ,
       -0.05939298,  0.13216337,  0.04035111,  0.19337628,  0.16649161,
       -0.28145108,  0.00913418, -0.08060089, -0.02234066,  0.1194856 ,
       -0.09446159, -0.01187711,  0.04454671,  0.01377404,  0.19575966,
       -0.05676382,  0.04761382,  0.05132244, -0.25285748, -0.05233306,
        0.01022195,  0.05693071,  0.22873623, -0.30330892,  0.01772877,
       -0.17236061,  0.51334351,  0.10912371,  0.07961901, -0.13171821,
        0.16055372,  0.18369018, -0.05771152, -0.02479237,  0.20080108,
        0.18622582,  0.13149338, -0.10864402, -0.01280621, -0.08

In [ ]:
На выходе должен получиться wordvec_df.shape = (49159, 200).

# Simple chat-bot example

In [173]:
# !pip install pymorphy2
# ! pip install stop_words
# !pip install annoy
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [176]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [177]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [178]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

In [179]:
sentences = [i for i in sentences if len(i) > 2]

In [180]:
sentences[0]

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']

In [181]:
modelW2V = Word2Vec(sentences=sentences, size=50, min_count=1, window=5)

In [182]:
modelFT = FastText(sentences=sentences, size=50, min_count=1, window=5)

In [184]:
w2v_index = annoy.AnnoyIndex(50 ,'angular')
ft_index = annoy.AnnoyIndex(50 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(50)
        vector_ft = np.zeros(50)
        for word in question:
            if word in modelW2V:
                vector_w2v += modelW2V[word]
                n_w2v += 1
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

True

In [185]:
w2v_index.get_nns_by_vector(np.zeros(50), 3)

[1577, 2139, 4363]

In [186]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(50)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [187]:
TEXT = "когда сменится президент"

In [188]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n',
 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n',
 'Твой.. \n']

In [189]:
get_response(TEXT, ft_index, modelFT, index_map)

['Следите за новостями в Украине. Как только там начнётся - считайте, что караван верблюдов уже вышел.. . Ждите, когда он прийдёт к вам.. . А лучше - скупайте баксы.... \n',
 'Поторопился..... \n',
 'Казалось бы, те усилия, что были приложены державами-победительницами в 1 Мировой войне для создания нового миропорядка, должны были обеспечить мир и благоденствие на многие годы. <br>Благие пожелания.. . <br>Рухнула Австро-Венгерская империя - "лоскутная империя", как её справедливо называли. На её обломках возникли как жизнеспособные государственные образования : Австрия, Венгрия, Чехословакия, Польша, так и новое "лоскутное государство" - Югославия. <br>Если щедрая раздача мандатов на управление бывшими германскими колониями в Африке и на Тихом океане не принесла эксцессов, то бывшие турецкие владения на Ближнем Востоке принесли бесконечную головную боль их новым владельцам. <br>Едва были заключены Вашингтонские соглашения об ограничении морских вооружений, как Япония, Германия и Италия

In [193]:
def get_response(size, question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(size)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [196]:
def CHECK(size,min_count,window):
    modelW2V = Word2Vec(sentences=sentences, size=size, min_count=min_count, window=window)
    modelFT = FastText(sentences=sentences, size=size, min_count=min_count, window=window)

    w2v_index = annoy.AnnoyIndex(size ,'angular')
    ft_index = annoy.AnnoyIndex(size ,'angular')

    index_map = {}
    counter = 0

    with open("prepared_answers.txt", "r") as f:
        for line in tqdm_notebook(f):
            n_w2v = 0
            n_ft = 0
            spls = line.split("\t")
            index_map[counter] = spls[1]
            question = preprocess_txt(spls[0])

            vector_w2v = np.zeros(size)
            vector_ft = np.zeros(size)
            for word in question:
                if word in modelW2V:
                    vector_w2v += modelW2V[word]
                    n_w2v += 1
                if word in modelFT:
                    vector_ft += modelFT[word]
                    n_ft += 1
            if n_w2v > 0:
                vector_w2v = vector_w2v / n_w2v
            if n_ft > 0:
                vector_ft = vector_ft / n_ft
            w2v_index.add_item(counter, vector_w2v)
            ft_index.add_item(counter, vector_ft)

            counter += 1

            if counter > 100000:
                break

    w2v_index.build(10)
    ft_index.build(10)

    w2v_index.get_nns_by_vector(np.zeros(size), 3)
    
    print(f"size={size}, min_count={min_count}, window={window}")
    print(get_response(size, TEXT, w2v_index, modelW2V, index_map))
    print(get_response(size, TEXT, ft_index, modelFT, index_map))

In [198]:
CHECK(size=100,min_count=1,window=5)

size=100, min_count=1, window=5
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', 'Твой.. \n']
['Следите за новостями в Украине. Как только там начнётся - считайте, что караван верблюдов уже вышел.. . Ждите, когда он прийдёт к вам.. . А лучше - скупайте баксы.... \n', 'БОРИСОВ <br>БОРИН БОРИСЕВИЧ БОРИСЕНКО БОРИСИХИН БОРИСКИН БОРИСОВ БОРИСОВЕЦ БОРИСОВИЧ БОРИСЯК БОРИЧЕВ БОРИЩЕВ БОРИЩЕНКО БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ <br> <br>Фамилия образована из отчества от крестильного имени Борис и его производных форм - Боря, Борище. По поводу объяснения самого имени нет единого мнения, в словарях предлагается объяснение из рус. и болг. - борец или как сокращенная форма славянского имени Борислав. Борисовец. От именвания жителя г. Борисов. Борисихин. может быть от Борисиха, жена Бориса.<br>. \n', 'Отели Хилтон, по всему миру. <br>К настояще

In [197]:
CHECK(size=100,min_count=1,window=1)

size=100, min_count=1, window=1
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', 'Постоянно тупить). \n']
['Отели Хилтон, по всему миру. <br>К настоящему времени Hilton представляет собой огромную корпорацию со штаб–квартирой в Беверли- Хилз (Калифорния, США) , которая в свою очередь состоит из восьми гостиничных цепей (Conrad Hotels, Doubletree, Embassy Suites Hotels, Hampton Inn, Hampton Inn and Suites, Hilton Hotels, Hilton Garden inn, Homewood Suites by Hilton), имеющих собственные стандарты обслуживания. В собственности и под управлением корпорации находится свыше 2,5 тыс. клубов и центров отдыха, отелей (свыше 500 на 148 000 номеров) на территории 75 стран пяти континентов и около 2 тыс. игорных заведений и букмекерских контор, располагающих более чем 600-тысячной клиентурой в 160 странах мира. Количество сотрудников — свыше 70 тыс. человек. Финансовый оборот — около 9 м

In [199]:
CHECK(size=50,min_count=1,window=5)

size=50, min_count=1, window=5
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', 'смотря на какую должность. \n']
['Следите за новостями в Украине. Как только там начнётся - считайте, что караван верблюдов уже вышел.. . Ждите, когда он прийдёт к вам.. . А лучше - скупайте баксы.... \n', 'Отели Хилтон, по всему миру. <br>К настоящему времени Hilton представляет собой огромную корпорацию со штаб–квартирой в Беверли- Хилз (Калифорния, США) , которая в свою очередь состоит из восьми гостиничных цепей (Conrad Hotels, Doubletree, Embassy Suites Hotels, Hampton Inn, Hampton Inn and Suites, Hilton Hotels, Hilton Garden inn, Homewood Suites by Hilton), имеющих собственные стандарты обслуживания. В собственности и под управлением корпорации находится свыше 2,5 тыс. клубов и центров отдыха, отелей (свыше 500 на 148 000 номеров) на территории 75 стран пяти континентов и около 2 тыс. игорны

In [200]:
CHECK(size=50,min_count=2,window=5)

size=50, min_count=2, window=5
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', 'Потому что большинство считали что это фарс чтобы подорвать репутацию новой власти Украины. Поэтому ему просто не сообщили место и день регистрации. И конечно потому что он был бы серьезным конкурентом на пост президента, его бы поддержала практически вся молодёжь. Но теперь он собирается стать мэром города Одессы, и в него есть все шансы. Дарт Вейдер выступает как интернет партия Украины, и ничего никому не обещает.. \n']
['БОРИСОВ <br>БОРИН БОРИСЕВИЧ БОРИСЕНКО БОРИСИХИН БОРИСКИН БОРИСОВ БОРИСОВЕЦ БОРИСОВИЧ БОРИСЯК БОРИЧЕВ БОРИЩЕВ БОРИЩЕНКО БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ <br> <br>Фамилия образована из отчества от крестильного имени Борис и его производных форм - Боря, Борище. По поводу объяснения самого имени нет единого мнения, в словарях предла

In [201]:
CHECK(size=50,min_count=3,window=5)

size=50, min_count=3, window=5
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', 'Твой.. \n']
['БОРИСОВ <br>БОРИН БОРИСЕВИЧ БОРИСЕНКО БОРИСИХИН БОРИСКИН БОРИСОВ БОРИСОВЕЦ БОРИСОВИЧ БОРИСЯК БОРИЧЕВ БОРИЩЕВ БОРИЩЕНКО БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ <br> <br>Фамилия образована из отчества от крестильного имени Борис и его производных форм - Боря, Борище. По поводу объяснения самого имени нет единого мнения, в словарях предлагается объяснение из рус. и болг. - борец или как сокращенная форма славянского имени Борислав. Борисовец. От именвания жителя г. Борисов. Борисихин. может быть от Борисиха, жена Бориса.<br>. \n', 'Жиды запретили обязательную государственную идеологию, но зато рекламируют и обязательно внедряют в жизнь граждан свою частную идеологию - ПОТРЕБЛЕНИЯ любой ценой, с отрицанием морали и совести, по Шекспиру - сделал д

In [202]:
CHECK(size=50,min_count=1,window=10)

size=50, min_count=1, window=10
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', '<p>Президент Турецкой Республики (тур. Türkiye Cumhuriyeti Cumhurbaşkanı) является главой государства Турецкой Республики.</p> <p>Являясь в основном церемониальной должностью, президент представляет Турецкую Республику и символизирует единство турецкой нации. При этом его важнейшими функциями является обеспечение исполнения Конституции и гармоничного взаимодействия других государственных органов. Статьи от 101 до 106 Конституции устанавливает требования к кандидатам, порядок избрания, обязанности и ответственность президента.</p> <p>После подписания 24 июля 1923 года Лозаннского мирного договора и международного признания Великого Национального Собрания Турции, 29 октября 1923 года было провозглашено создание Турецкой Республики, являвшейся преемницей Османской империи; в тот же день первым прези

In [203]:
CHECK(size=50,min_count=2,window=10)

size=50, min_count=2, window=10
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', '<p>Президент Турецкой Республики (тур. Türkiye Cumhuriyeti Cumhurbaşkanı) является главой государства Турецкой Республики.</p> <p>Являясь в основном церемониальной должностью, президент представляет Турецкую Республику и символизирует единство турецкой нации. При этом его важнейшими функциями является обеспечение исполнения Конституции и гармоничного взаимодействия других государственных органов. Статьи от 101 до 106 Конституции устанавливает требования к кандидатам, порядок избрания, обязанности и ответственность президента.</p> <p>После подписания 24 июля 1923 года Лозаннского мирного договора и международного признания Великого Национального Собрания Турции, 29 октября 1923 года было провозглашено создание Турецкой Республики, являвшейся преемницей Османской империи; в тот же день первым прези

In [204]:
CHECK(size=50,min_count=2,window=20)

size=50, min_count=2, window=20
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', 'Это тост.... \n']
['двумя руками - за. \n', 'Ну конечтно это мой любимый фильм. Но все равно кажется что продолжение есть продолжение. Никто не сыграет так как сыграл А. Мягков. Верно?. \n', 'БОРИСОВ <br>БОРИН БОРИСЕВИЧ БОРИСЕНКО БОРИСИХИН БОРИСКИН БОРИСОВ БОРИСОВЕЦ БОРИСОВИЧ БОРИСЯК БОРИЧЕВ БОРИЩЕВ БОРИЩЕНКО БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ <br> <br>Фамилия образована из отчества от крестильного имени Борис и его производных форм - Боря, Борище. По поводу объяснения самого имени нет единого мнения, в словарях предлагается объяснение из рус. и болг. - борец или как сокращенная форма славянского имени Борислав. Борисовец. От именвания жителя г. Борисов. Борисихин. может быть от Борисиха, жена Бориса.<br>. \n']


In [205]:
CHECK(size=50,min_count=2,window=20)

size=50, min_count=2, window=20
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', 'Твой.. \n']
['БОРИСОВ <br>БОРИН БОРИСЕВИЧ БОРИСЕНКО БОРИСИХИН БОРИСКИН БОРИСОВ БОРИСОВЕЦ БОРИСОВИЧ БОРИСЯК БОРИЧЕВ БОРИЩЕВ БОРИЩЕНКО БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ <br> <br>Фамилия образована из отчества от крестильного имени Борис и его производных форм - Боря, Борище. По поводу объяснения самого имени нет единого мнения, в словарях предлагается объяснение из рус. и болг. - борец или как сокращенная форма славянского имени Борислав. Борисовец. От именвания жителя г. Борисов. Борисихин. может быть от Борисиха, жена Бориса.<br>. \n', 'Я и так успешна и независима. А стать участником - не хочу.. \n', 'Гарри женился на Невиле Долгопупсе. Дамблдор на Волан-де морте)). \n']


In [206]:
CHECK(size=10,min_count=1,window=5)

size=10, min_count=1, window=5
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', '<p>Президент Турецкой Республики (тур. Türkiye Cumhuriyeti Cumhurbaşkanı) является главой государства Турецкой Республики.</p> <p>Являясь в основном церемониальной должностью, президент представляет Турецкую Республику и символизирует единство турецкой нации. При этом его важнейшими функциями является обеспечение исполнения Конституции и гармоничного взаимодействия других государственных органов. Статьи от 101 до 106 Конституции устанавливает требования к кандидатам, порядок избрания, обязанности и ответственность президента.</p> <p>После подписания 24 июля 1923 года Лозаннского мирного договора и международного признания Великого Национального Собрания Турции, 29 октября 1923 года было провозглашено создание Турецкой Республики, являвшейся преемницей Османской империи; в тот же день первым презид

In [207]:
CHECK(size=10,min_count=1,window=5)

size=10, min_count=1, window=5
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', 'Они еще и дебилы-тунеядцы-вредители..... \n']
['Жиды запретили обязательную государственную идеологию, но зато рекламируют и обязательно внедряют в жизнь граждан свою частную идеологию - ПОТРЕБЛЕНИЯ любой ценой, с отрицанием морали и совести, по Шекспиру - сделал дельце - заметай следы. Или кто смел, тот и съел. Поэтому страна добилась 200 евреев- жидов, правящих Россией.... \n', 'Следите за новостями в Украине. Как только там начнётся - считайте, что караван верблюдов уже вышел.. . Ждите, когда он прийдёт к вам.. . А лучше - скупайте баксы.... \n', 'Опять майдан!. \n']


In [208]:
CHECK(size=10,min_count=2,window=5)

size=10, min_count=2, window=5
['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n', 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n', '<p> Университеты, театр,библиотеки,музеи, относительно независимая пресса - эпоха Просвещения-вторая половина 18 века </p>. \n']
['БОРИСОВ <br>БОРИН БОРИСЕВИЧ БОРИСЕНКО БОРИСИХИН БОРИСКИН БОРИСОВ БОРИСОВЕЦ БОРИСОВИЧ БОРИСЯК БОРИЧЕВ БОРИЩЕВ БОРИЩЕНКО БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ БОРИСЕНОК БОРИСЕНКОВ БОРИСОВЕЦ БОРИСЫЧЕВ <br> <br>Фамилия образована из отчества от крестильного имени Борис и его производных форм - Боря, Борище. По поводу объяснения самого имени нет единого мнения, в словарях предлагается объяснение из рус. и болг. - борец или как сокращенная форма славянского имени Борислав. Борисовец. От именвания жителя г. Борисов. Борисихин. может быть от Борисиха, жена Бориса.<br>. \n', 'Видео должно принадлежать только тебе, и не вставляй в видео какие нибудь песни музыку, все только с

### Экспортируем лучшие модели

In [209]:
filename = 'modelW2V_best'
outfile = open(filename,'wb')
pickle.dump(modelW2V,outfile)
outfile.close()

In [210]:
filename = 'modelFT_best'
outfile = open(filename,'wb')
pickle.dump(modelFT,outfile)
outfile.close()